# 1. Подготовка

In [1]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import re
import torch
import transformers
from nltk.corpus import stopwords as stop_words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier

In [2]:
#comments = pd.read_csv('C:/Users/vladi/Desktop/toxic_comments.csv')
comments = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
comments.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Перед обучением моделей данные нужно преобразовать. Комментарии нужно лемматизировать, также необходимо убрать знаки препинания.

In [4]:
#функции лемматизации и очистки текста
#тут, насколько я понял, WordNetLemmatizer не умеет работать с предложениями, и ему надо подать на вход список слов
#с указанной частью речи (глагол, существительное и т.д.). Поскольку код части речи nltk возвращает в виде,
#отличающемся от того, что принимает WordNetLemmatizer, нужно этот код поменять. Это делает первая функция.
#Далее вторая функция берет предложение, делит его на слова, каждому слову присваивает код части речи, передает в 
#WordNetLemmatizer и из полученных ответов собирает предложение обратно.

lemmatizer = WordNetLemmatizer()
def nltk2wn_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:                    
        return None
    
def lemmatize_sentence(sentence):
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))    
    wn_tagged = map(lambda x: (x[0], nltk2wn_tag(x[1])), nltk_tagged)
    res_words = []
    for word, tag in wn_tagged:
        if tag is None:                        
            res_words.append(word)
        else:
            res_words.append(lemmatizer.lemmatize(word, tag))
    return " ".join(res_words)

def clear_text(text):
    cleared = re.sub(r'[^a-zA-Z0-9]', ' ', text)
    cleared = " ".join(cleared.split())
    return cleared

In [5]:
%%time
comments['lemmatized'] = comments['text'].apply(lemmatize_sentence)
comments['cleared'] = comments['lemmatized'].apply(clear_text)

Wall time: 14min 10s


In [6]:
comments.head()

,text,toxic,lemmatized,cleared
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits make under my userna...,Explanation Why the edits make under my userna...
1,D'aww! He matches this background colour I'm s...,0,D'aww ! He match this background colour I 'm s...,D aww He match this background colour I m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,"Hey man , I 'm really not try to edit war . It...",Hey man I m really not try to edit war It s ju...
3,"""\nMore\nI can't make any real suggestions on ...",0,`` More I ca n't make any real suggestion on i...,More I ca n t make any real suggestion on impr...
4,"You, sir, are my hero. Any chance you remember...",0,"You , sir , be my hero . Any chance you rememb...",You sir be my hero Any chance you remember wha...


# 2. Обучение

In [7]:
#стоп-слова
stop_words = set(stop_words.words('english'))

In [8]:
#разбиение на обучающую и тестовую выборки
train, valid = train_test_split(comments, test_size=0.25, random_state=12345)

In [9]:
#модель TF-IDF
tf_idf_model = TfidfVectorizer(stop_words=stop_words)

corpus_train = train['cleared']
target_train = train['toxic']
corpus_valid = valid['cleared']
target_valid = valid['toxic']

train_feature = tf_idf_model.fit_transform(corpus_train)
test_feature = tf_idf_model.transform(corpus_valid)

In [10]:
#модель логистической регрессии
model_linear = LogisticRegression()
model_linear.fit(train_feature, target_train)

C:\Users\vladi\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [13]:
predictions_linear = model_linear.predict(test_feature)
score_linear = f1_score(target_valid, predictions_linear)
print('F1-мера логистической регрессии, построенной на признаках TF-IDF:', score_linear)

F1-мера логистической регрессии, построенной на признаках TF-IDF: 0.7363462387752098


In [14]:
%%time
#модель LGBMClassifier
model_lgbm = LGBMClassifier(random_state=12345)
model_lgbm.fit(train_feature, target_train)
predictions_lgbm = model_lgbm.predict(test_feature)
score_lgbm = f1_score(target_valid, predictions_lgbm)
print('F1-мера модели классификации LGBM:', score_lgbm)

F1-мера модели классификации LGBM: 0.7468336416678526
Wall time: 23.9 s


# 3. Выводы

В рамках проекта была построена модель машинного обучения, классифицирующая комментарии на токсичные и нетоксичные.

1. К исходным комментариям была применена лемматизация, а также при помощи регулярных выражений выполнена очистка комментариев от знаков препинания.
2. После лемматизации и очистки была выполнена векторизация TF-IDF.
3. На полученных в результате векторизации признаках были оучены модели логистической регрессии и LGBM-классификатора.

**Выводы.**
1. Самый долгий процесс - лемматизация текстов.
2. Логистическая регрессия показывает достаточно высокое значение F1-меры. При этом работает быстро.
3. Требуемый результат дает LGBM-классификатор с параметрами по умолчанию. Путем подбора гиперпараметров можно оптимизировать эту модель, добившись более высокой точности.
4. Применение модели BERT при кодировке текстов позволит добиться более точной работы модели на новых текстах, т.к. BERT учитывает связь между словами.